In [124]:
import nltk
import pandas as pd
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import string
import matplotlib.pyplot as plt
%matplotlib inline
ps = nltk.PorterStemmer()
from nltk.tokenize import sent_tokenize, word_tokenize
from pprint import pprint

from string import punctuation

nltk.download('punkt')


# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
  

[nltk_data] Downloading package punkt to /Users/selma/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [125]:
tweets=pd.read_csv('Data_Science.csv', sep='\t')

In [126]:
tweets.head()

,Unnamed: 0,Favorite Count,Favorited,ID,Phone Type,Replied,Retweet Count,Retweeted,Screen Name,Tweet Coordinates,Tweet Created At,Tweet Text,User Location,User Name
0,0,0,False,1171978318368796672,Twitter for iPhone,1.171975e+18,0,False,KingoOsel,NaN,2019-09-12 02:46:20,@jagare_ulrika @KirkDBorne @Strategy_Gal The b...,NaN,Kingo Osel
1,1,0,False,1171978274978848770,rstatsretweetingtool,NaN,1,False,rstatstweet,NaN,2019-09-12 02:46:09,RT @seanrife: Hello science tweeps! I am worki...,NaN,Rstats
2,2,0,False,1171978082988646400,Twitter for iPad,NaN,1,False,follysantidote,NaN,2019-09-12 02:45:23,RT @3AInstitute: We are so excited to share at...,"Queensland, Australia",Folly's Antidote
3,3,0,False,1171978001594159105,Wynand 2.0,NaN,20,False,wynandbooysen,NaN,2019-09-12 02:45:04,RT @gp_pulipaka: Data Science for Good: Recomm...,South Africa,🇿🇦 Wynand Booysen 💻 改善
4,4,0,False,1171977936150417409,ComaBot,NaN,20,False,Comatose_D,NaN,2019-09-12 02:44:48,RT @gp_pulipaka: Data Science for Good: Recomm...,Game Grid,comatosed


In [127]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
Unnamed: 0           5000 non-null int64
Favorite Count       5000 non-null int64
Favorited            5000 non-null bool
ID                   5000 non-null int64
Phone Type           5000 non-null object
Replied              359 non-null float64
Retweet Count        5000 non-null int64
Retweeted            5000 non-null bool
Screen Name          5000 non-null object
Tweet Coordinates    6 non-null object
Tweet Created At     5000 non-null object
Tweet Text           5000 non-null object
User Location        3941 non-null object
User Name            5000 non-null object
dtypes: bool(2), float64(1), int64(4), object(7)
memory usage: 478.6+ KB


# Feature engeneering 

In [128]:
# Number of words
tweets ['word_count']= tweets['Tweet Text'].apply(lambda x: len(str(x).split(" ")))
# Number of Characters 
tweets['char_count'] = tweets['Tweet Text'].str.len()
# average word length
def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/len(words))

tweets['avg_word'] = tweets['Tweet Text'].apply(lambda x: avg_word(x))
# Count Stopwords
from nltk.corpus import stopwords
stop = stopwords.words('english')


tweets['stopwords_counts'] = tweets['Tweet Text'].apply(lambda x: len([x for x in x.split() if x in stop]))
tweets['hastags'] = tweets['Tweet Text'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
tweets['tags'] = tweets['Tweet Text'].apply(lambda x: len([x for x in x.split() if x.startswith('@')]))
tweets['numerics'] = tweets['Tweet Text'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))





# Clean tweet text to use in Machine learning

In [129]:
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [word for word in tokens if word not in stop]
    text= [word for word in text  if not word.startswith('#')]
    text=[word for word in text if not word.startswith ('@')]
    text=[word for word in text if word!='rt']
    text=[re.sub(r"http\S+", "", word) for word in text]
    return text

<>:3: DeprecationWarning: invalid escape sequence \W
<>:3: DeprecationWarning: invalid escape sequence \W
<>:3: DeprecationWarning: invalid escape sequence \W
<ipython-input-129-9cca8244e1eb>:3: DeprecationWarning: invalid escape sequence \W
  tokens = re.split('\W+', text)


In [130]:
tweets['Clean_tweet_text']= tweets['Tweet Text'].apply(clean_text)

In [131]:
tweets['Clean_tweet_text'].head()

0    [jagareulrika, kirkdborne, strategygal, books,...
1    [seanrife, hello, science, tweeps, working, rr...
2    [3ainstitute, excited, share, ellenbroad, join...
3    [gppulipaka, data, science, good, recommendati...
4    [gppulipaka, data, science, good, recommendati...
Name: Clean_tweet_text, dtype: object

# Bigrams, Trigrams ans Lemmatyization

In [132]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(tweets['Clean_tweet_text'], min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[tweets['Clean_tweet_text']], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[tweets['Clean_tweet_text'][0]]])

['jagareulrika', 'kirkdborne', 'strategygal', 'books', 'amp', 'series', 'fine', 'thought', 'would', 'hilarious', 'ha']


In [133]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [134]:
# Form Bigrams
tweets['bigrams'] = make_bigrams(tweets['Clean_tweet_text'])

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(tweets['bigrams'] , allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[0])

['jagareulrika', 'kirkdborne', 'strategygal', 'book', 'amp', 'series', 'fine', 'thought', 'would', 'hilarious']


# Create dictionary and corpus needed for topic modeling

In [135]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[0])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)]


In [136]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:10]]

[[('amp', 1),
  ('book', 1),
  ('fine', 1),
  ('hilarious', 1),
  ('jagareulrika', 1),
  ('kirkdborne', 1),
  ('series', 1),
  ('strategygal', 1),
  ('thought', 1),
  ('would', 1)],
 [('anyone', 1),
  ('begin', 1),
  ('collection', 1),
  ('comfortable', 1),
  ('datum', 1),
  ('rrr', 1),
  ('rstat', 1),
  ('science', 1),
  ('seanrife', 1),
  ('tweep', 1),
  ('work', 1)],
 [('3ainstitute', 1),
  ('addition', 1),
  ('ellen', 1),
  ('ellenbroad', 1),
  ('excited', 1),
  ('great', 1),
  ('institute', 1),
  ('join', 1),
  ('midoctober', 1),
  ('share', 1)],
 [('datum', 1),
  ('science', 1),
  ('ai', 1),
  ('analytic', 1),
  ('bigdata', 1),
  ('datascience', 1),
  ('good_recommendation', 1),
  ('gppulipaka', 1),
  ('machinelearne', 1),
  ('pytorch_p', 1),
  ('system', 1)],
 [('datum', 1),
  ('science', 1),
  ('ai', 1),
  ('analytic', 1),
  ('bigdata', 1),
  ('datascience', 1),
  ('good_recommendation', 1),
  ('gppulipaka', 1),
  ('machinelearne', 1),
  ('pytorch_p', 1),
  ('system', 1)],
 [('

# LDA Model

In [137]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [138]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.071*"datum" + 0.063*"science" + 0.039*"datascience" + 0.021*"ai" + '
  '0.018*"data" + 0.014*"bigdata" + 0.013*"machine" + 0.012*"amp" + '
  '0.012*"learning" + 0.011*"analytic"'),
 (1,
  '0.052*"datum" + 0.031*"science" + 0.014*"great" + 0.014*"new" + '
  '0.008*"comfort" + 0.008*"find_rare" + 0.008*"markets_gone" + '
  '0.008*"mad_investor" + 0.007*"open" + 0.007*"project"'),
 (2,
  '0.109*"datum" + 0.061*"science" + 0.058*"need" + 0.053*"big" + 0.048*"give" '
  '+ 0.047*"focus" + 0.047*"campaign" + 0.047*"collection" + 0.047*"leave" + '
  '0.047*"vote"'),
 (3,
  '0.069*"science" + 0.065*"datum" + 0.024*"learn" + 0.014*"data" + '
  '0.011*"use" + 0.009*"look" + 0.008*"machine" + 0.007*"course" + '
  '0.007*"work" + 0.007*"python"')]


In [139]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.559199649923401

Coherence Score:  0.3851360916853905


In [140]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

//anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.096599 -0.046148       1        1  28.829462
0     -0.093994 -0.192226       2        1  25.753534
1     -0.130627  0.219314       3        1  23.400663
2      0.321220  0.019060       4        1  22.016336, topic_info=     Category         Freq                  Term        Total  loglift  \
52    Default   630.000000                  need   630.000000  30.0000   
881   Default   574.000000                   big   574.000000  29.0000   
328   Default   519.000000                  give   519.000000  28.0000   
1336  Default   514.000000                 focus   514.000000  27.0000   
1333  Default   509.000000              campaign   509.000000  26.0000   
12    Default   509.000000            collection   509.000000  25.0000   
910   Default   508.000000                 leave   508.000000  24.0000   
1340  Default   507.000000                  vote   507.000000  23.0000   
1339  Default   507.000000                urgent   507.000000  22.0000   
1337  Default   507.000000                sudden   507.000000  21.0000   
1334  Default   507.000000               cumming   507.000000  20.0000   
1338  Default   506.000000             tomwatson   506.000000  19.0000   
1335  Default   504.000000              extremel   504.000000  18.0000   
34    Default   498.000000           datascience   498.000000  17.0000   
162   Default   348.000000                 learn   348.000000  16.0000   
31    Default   270.000000                    ai   270.000000  15.0000   
45    Default   433.000000                  data   433.000000  14.0000   
26    Default   166.000000                 great   166.000000  13.0000   
33    Default   173.000000               bigdata   173.000000  12.0000   
14    Default  3619.000000                 datum  3619.000000  11.0000   
650   Default   152.000000              learning   152.000000  10.0000   
155   Default   163.000000                   use   163.000000   9.0000   
340   Default   238.000000                   new   238.000000   8.0000   
150   Default   289.000000               machine   289.000000   7.0000   
0     Default   171.000000                   amp   171.000000   6.0000   
32    Default   137.000000              analytic   137.000000   5.0000   
92    Default   128.000000                  look   128.000000   4.0000   
198   Default   110.000000                 skill   110.000000   3.0000   
1282  Default    98.000000               comfort    98.000000   2.0000   
138   Default   105.000000                   get   105.000000   1.0000   
...       ...          ...                   ...          ...      ...   
910    Topic4   507.630798                 leave   508.401459   1.5119   
12     Topic4   508.298920            collection   509.070648   1.5119   
1340   Topic4   506.993408                  vote   507.763611   1.5119   
1339   Topic4   506.973083                urgent   507.743286   1.5119   
1337   Topic4   506.973083                sudden   507.743286   1.5119   
1334   Topic4   506.660950               cumming   507.431274   1.5119   
1338   Topic4   506.054016             tomwatson   506.824188   1.5119   
1335   Topic4   504.001831              extremel   504.772003   1.5119   
123    Topic4    93.511948                   phd    94.305702   1.5049   
124    Topic4    79.845558         says_manually    80.615746   1.5038   
122    Topic4    79.845558  gshotwell_definitely    80.615746   1.5038   
121    Topic4    79.845558              csv_file    80.615746   1.5038   
56     Topic4    23.594643            understand    24.528345   1.4746   
1733   Topic4    21.698427                 cloud    22.579750   1.4736   
978    Topic4    26.333670           engineering    27.525566   1.4691   
1855   Topic4    16.360458                  huge    17.249363   1.4605   
391    Topic4    22.573313                global    23.909033   1.4559   
1735   